In [11]:
import pdfplumber
import pymongo
import pandas as pd

In [12]:
def extract_dimensions(pdf_path):
    dimensions = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Look for tables in each page
            tables = page.extract_tables()
            for table in tables:
                # Process each table row into a dictionary
                headers = ["Dim", "Description", "Comment", "Tol (-)", "Tol (+)", "XS", "S", "M", "L", "XL"]
                for row in table:
                    if len(row) == len(headers):  # Ensure the row matches headers
                        row_data = dict(zip(headers, row))
                        dimensions.append(row_data)
    return dimensions


In [13]:
def store_in_mongo(data, db_name, collection_name):
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[db_name]
    collection = db[collection_name]
    
    # Insert data
    collection.insert_many(data)
    print(f"Inserted {len(data)} records into {collection_name} collection in {db_name} database.")


In [15]:
def main():
    pdf_path = r"E:\placement\jupiterBrothers\input_doc.pdf"  # Path to the input PDF document
    dimensions_data = extract_dimensions(pdf_path)
    print(dimensions_data)
    # MongoDB storage
    db_name = "technical_specs"
    collection_name = "dimensions"
    store_in_mongo(dimensions_data, db_name, collection_name)

if __name__ == "__main__":
    main()


[]


TypeError: documents must be a non-empty list